In [3]:
from officelib.xllib import *
from officelib import const as c
from hello.hello import *
import os
os.chdir("C:/.replcache")

def download(ip, name, fp):
    app = open_hello(ip)
    app.login()
    r = app.getdatareport_bybatchname(name)
    with open(fp, 'wb') as f:
        f.write(r)

def find_column(rng, header):
    return rng.Find(header, SearchOrder=c.xlByColumns)

def fudge_column(cells):
    """ For some reason, cells.Find() can't find 
    the requested header if the header occurs in the 
    first column, so insert a dummy here """
    cells.Columns(1).Insert()

def get_header_data(cells, h):
    col = find_column(cells, h)
    s = col.Offset(2,2)
    e = s.End(c.xlDown)
    rows = e.Row-s.Row + 1
    return col, s, e, rows

def make_col(ref, col_offset, header, row_offset):
    c = ref.Offset(1, col_offset)
    c.Value = header
    f = c.Offset(2, 1)
    s = f.Offset(row_offset, 1)
    return c, f, s

def create_data(cells):
    rng=cells.Rows(1)
    fo = get_data("FilterOvenPV(C)")
    ta = get_data("TempA(C)")
    tb = get_data("TempB(C)")
    
    amb = tb.Offset(1,4)
    diff = amb.Offset(1,2)
    
    amb.Value = "Ambient"
    diff.Value = "Diff A-B"
    
    return fo, ta, tb, amb, diff

def make_amb_diff(cells, offset=1):
    fo, fo_start, fo_end, fo_r = get_header_data(cells, "FilterOvenPV(C)")
    tempa, tempa_start, tempa_end, tempa_r = get_header_data(cells, "TempA(C)")
    tempb, tempb_start, tempb_end, tempb_r = get_header_data(cells, "TempB(C)")

    assert (fo_r == tempa_r and fo_r == tempb_r)
    
    amb, amb_first, amb_start = make_col(tempb, 4, "Ambient", offset)
    diff, diff_first, diff_start = make_col(amb, 2, "Diff A-B", offset)

    amb_form = "=%s" % addr(fo_start)
    diff_form = "=%s-%s" % (addr(tempa_start), addr(tempb_start))

    amb_first.Value = amb_form
    diff_first.Value = diff_form
    
    form_rng = cells.Range(amb_first, diff_first)
    dst = cells.Range(amb_first, diff_first.Offset(fo_r, 1))
    form_rng.AutoFill(dst)
    
    return amb_start, diff_start

def addr(r):
    return r.GetAddress(False, False)

def chart_data_rng(cells, xc, yc):
    x = cells.Range(xc, xc.End(c.xlDown))
    y = cells.Range(yc, yc.End(c.xlDown))
    return x, y

def min_and_max(cells, x_start, ystart):
    rng = cells.Range(x_start, x_start.End(c.xlDown))
    yrng = cells.Range(ystart, ystart.End(c.xlDown))
    x = [x1[0] for x1 in rng.Value]
    y = [y1[0] for y1 in yrng.Value]
    xmin = min(x)
    xmax = max(x)
    ymin = min(y)
    ymax = max(y)
    return xmin, xmax, ymin, ymax

def shrink_markers(c, sz=2):
    for s in c.SeriesCollection():
        s.MarkerSize=sz

def analyze_rtd_data(pth, title, offset=1):
    
    fp = os.path.abspath(pth)
    
    # Init ctx
    xl = Excel(visible=False)
    with HiddenXl(xl):
        wb = xl.Workbooks.Open(fp)
        ws = wb.Worksheets(1)
        cells = ws.Cells  
        _analyze_rtd_data_inner(xl, wb, ws, cells, title, offset)

def _analyze_rtd_data_inner(xl, wb, ws, cells, title, offset):
    # data analysis 
    fudge_column(cells)
    amb_start, diff_start = make_amb_diff(cells, offset)
    
    # create chart ambient vs rtd error
    c1 = CreateChart(ws)
    x, y = chart_data_rng(cells, amb_start, diff_start)
    series = CreateDataSeries(c1, x, y)
    
    FormatChart(c1, None, title, "Ambient Temp(C)", "Err A-B", True, False)
    shrink_markers(c1)
    
    xmin, xmax, ymin, ymax = min_and_max(cells, amb_start, diff_start)
    FormatAxesScale(c1, int(xmin)-2, int(xmax)+2, round(ymin-.2, 1), round(ymax+.2, 1))
        
    c1.Location(c.xlLocationAsNewSheet, "AmbRTD")
    c1 = wb.Charts("AmbRTD")
    
    # create chart amb v. time and diff v. time
    
    # time data
    tb = find_column(cells, "TempB")
    ts = tb.Offset(amb_start.Row,1)
    te = ts.End(c.xlDown)
    t = cells.Range(ts, te)
    
    c2 = CreateChart(ws)
    s2 = CreateDataSeries(c2, t, x, "Ambient")
    s3 = CreateDataSeries(c2, t, y, "Diff")
    FormatChart(c2, None, title, "Time", "Temp(C)")
    s3.AxisGroup = 2
    y2ax = c2.Axes(2, c.xlSecondary); y2ax.HasTitle=True; y2ax.AxisTitle.Text = "Diff A-B"
    
    xmin, xmax, y1min, y1max = min_and_max(cells, ts, amb_start)
    _, _, y2min, y2max = min_and_max(cells, ts, diff_start)
    y1min = int(y1min-1)
    y1max = int(y1max+1)
    y2min = round(y2min-.2,1)
    y2max = round(y2max+.2,1)
    FormatAxesScale(c2, None, None, y1min, y1max, y2min, y2max)    
    shrink_markers(c2)
    c2.Axes(1).TickLabels.NumberFormat = "[$-409]h:mm AM/PM;@"
    c2.Location(c.xlLocationAsNewSheet, "vs Time")
    return c1, c2

In [13]:
def download_and_analyze(ip, name, fp, title, offset=1):
    if not os.path.exists(fp):
        download(ip, name, fp)
    analyze_rtd_data(fp, title, offset)

In [14]:
#from datetime import datetime as dt
#os.chdir("C:/.replcache")
#name='rtdtest'
#fp = name + " " + dt.now().strftime("%y%m%d%H%M") + ".csv"
#download_and_analyze('71.189.82.196:6', name, fp, '15L Ambient RTD Effect', 150)

In [15]:
from datetime import datetime as dt
os.chdir("C:/.replcache")
name='rtdtestnoback'
fp = name + " " + dt.now().strftime("%y%m%d%H%M") + ".csv"
download_and_analyze('71.189.82.196:6', name, fp, '15L Ambient RTD Effect, No Back', 1)

In [4]:
def animate_chart(step=5):
    xl = Excel()
    wb = xl.ActiveWorkbook
    ws = wb.Worksheets(1)
    cells = ws.Cells
    cr = cells.Range

    try:
        ch = wb.Charts("Test")
    except: 
        ch = CreateChart(ws)
        ch.Location(c.xlLocationAsNewSheet, "Test")
        ch = wb.Charts("Test")

    xtop = cr("K2")
    ytop = cr("L2")
    dtop = cr("H2")
    
    def get_date(c, r):
        return c.Offset(r, 1).Value.strftime("%I:%M%p %m/%d/%y").lower()

    def get_range(c, r):
        return cr(c, c.Offset(r, 1))

    PurgeSeriesCollection(ch)
    end_row = xtop.End(c.xlDown).Row
    x = get_range(xtop, 1)
    y = get_range(ytop, 1)
    s=CreateDataSeries(ch, x, y)
    shrink_markers(ch)
    FormatChart(ch, Trendline=True)

    for i in range(step, end_row-1, step):
        x = get_range(xtop, i)
        y = get_range(ytop, i)
        s.XValues = x
        s.Values = y
        print("\r", i, "/", end_row, get_date(dtop, i), end="")
    i = end_row - 1
    x = get_range(xtop, i)
    y = get_range(ytop, i)
    s.XValues = x
    s.Values = y
    print("\r", i+1, "/", end_row, get_date(dtop, i), end="")

In [6]:
animate_chart(1)

 1398 / 1398 04:46pm 05/19/16